In [7]:
import pandas as pd

In [ ]:
df_month_targets = pd.read_csv('data/month_targets.csv')


,product_code,brand_cat
0,21474,Test
1,11769,Test
2,17007,Kapiva
3,22188,Kapiva
4,23676,Kapiva


In [9]:
df_bt = df_bt.reset_index()
df_bt.head()

,Month,brand_cat,brand_sale_range,%applied
0,Feb-25,Kapiva,0-100,2
1,Feb-25,Kapiva,100-200,4
2,Feb-25,Kapiva,200-300,6
3,Feb-25,Kapiva,300-400,8
4,Feb-25,Kapiva,400-500,10


In [10]:
merged_df = df_brands.merge(df_bt, on="brand_cat", how="left")

# Rename product_code to product_id
merged_df.rename(columns={"product_code": "product_id"}, inplace=True)

# Reorder columns as per requirement
final_df = merged_df[["product_id", "brand_cat", "Month", "brand_sale_range", "%applied"]]

# Save the result to a CSV file
final_df.to_csv("merged_output.csv", index=False)

print(final_df)

     product_id brand_cat   Month brand_sale_range  %applied
0         21474      Test  Jul-24            0-100         2
1         21474      Test  Jul-24          100-200         4
2         21474      Test  Jul-24          200-300         6
3         21474      Test  Jul-24          300-400         8
4         21474      Test  Jul-24          400-500        10
..          ...       ...     ...              ...       ...
109       11769      Test  Jul-24          300-400         8
110       11769      Test  Jul-24          400-500        10
111       11769      Test  Jul-24         500-1000        12
112       11769      Test  May-24         100-2000         6
113       11769      Test  May-24     2000-2000000        12

[114 rows x 5 columns]


In [6]:
final_df.head()

,product_id,brand_cat,Month,brand_sale_range,%applied
0,21474,Test,Jul-24,0-100,2
1,21474,Test,Jul-24,100-200,4
2,21474,Test,Jul-24,200-300,6
3,21474,Test,Jul-24,300-400,8
4,21474,Test,Jul-24,400-500,10


In [ ]:
# from main.incentive_leaderboard_report_qty import generate_il_report
# from main.incentive_leaderboard_report_range import generate_il_report_range
# from main.stores_month_targets import generate_stores_month_targets
# from main.spot_targets_reports import generate_stores_spot_targets
# from main.advanced_urgent_reports import generate_au_reports
# from main.wow_reports import generate_wow_reports
# from main.zero_brand_sales import generate_zero_brand_report


# from utils.db_utils import save_csv_to_local, store_data_to_local, fetch_all_tables
# from utils.transform_remote import take_requried_columns
# import pandas as pd



# ## Saving the required data to local database
# # users, products, stores, sales_invoices, sales_invoice_details, sales_return, sales_return_details, advance_sales_invoices= fetch_all_tables()
# # users, products, stores, sales_invoices, sales_invoice_details, sales_return, sales_return_details, advance_sales_invoices= take_requried_columns(users, products, stores, sales_invoices, sales_invoice_details, sales_return, sales_return_details,advance_sales_invoices)
# # store_data_to_local("users", users)
# # store_data_to_local("products", products)
# # store_data_to_local("stores", stores)
# # store_data_to_local("sales_invoices", sales_invoices)
# # store_data_to_local("sales_invoice_details", sales_invoice_details)
# # store_data_to_local("sales_return", sales_return)
# # store_data_to_local("sales_return_details", sales_return_details)
# # store_data_to_local("advance_sales_invoices", advance_sales_invoices)
# # print("Data stored in local database")


# # REPORT 1
# # save_csv_to_local("./brand_tieup_1.csv")
# # final_ilr = generate_il_report()
# # store_data_to_local("incentive_leaderboard_report_qty", final_ilr)


# # REPORT 2
# # Adding the data to the local database
# # save_csv_to_local("./brand_tieup_2.csv")
# # save_csv_to_local("./brands.csv")

# # final_ilr = generate_il_report_range()
# # store_data_to_local("incentive_leaderboard_report_range", final_ilr)


# # REPORT 3
# # save_csv_to_local("./month_targets.csv")
# # test = generate_stores_month_targets()
# # store_data_to_local("sales_target_report", test)


# # ## REPORT 4

# # ## Fetching data from remote and then storing it in local
# # save_csv_to_local("./spot_targets.csv")
# # generate_stores_spot_targets()
# # store_data_to_local("spot_targets_report", generate_stores_spot_targets())

# # ### REPORT 5
# # x = generate_au_reports()
# # store_data_to_local("advanced_urgent_report", x)


# # ### REPORT 6
# # generate_wow_reports()\


# ## Report 7
# x = generate_zero_brand_report()
# store_data_to_local("zero_brand_sales", x)
# print("Data stored in local database")

# test.py

In [ ]:
from flask import Blueprint, render_template, request, send_file, jsonify
from werkzeug.utils import secure_filename
import os
import pandas as pd
from app import cache

from main.incentive_leaderboard_report_qty import generate_il_report
from main.incentive_leaderboard_report_range import generate_il_report_range
from main.stores_month_targets import generate_stores_month_targets
from main.spot_targets_reports import generate_stores_spot_targets
from main.advanced_urgent_reports import generate_au_reports
from main.wow_reports import generate_wow_reports
from main.zero_brand_sales import generate_zero_brand_report
from utils.db_utils import save_csv_to_local, store_data_to_local, read_local_data

bp = Blueprint('reports', __name__)

UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'csv'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@bp.route('/')
def index():
    return render_template('index.html')

@bp.route('/report/incentive-qty/upload', methods=['POST'])
def incentive_qty_upload():
    # 1. User uploads new data
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    if file and allowed_file(file.filename):
        # Reset cache here
        cache.clear()
        if not os.path.exists(UPLOAD_FOLDER):
            os.makedirs(UPLOAD_FOLDER)

        filename = secure_filename(file.filename)
        filepath = os.path.join(UPLOAD_FOLDER, filename)
        file.save(filepath)

        # 2. Save data to DB
        save_csv_to_local(filepath, 'brand_tieup_1')

        # 3. Generate entire report
        full_report = generate_il_report(filepath)  # Example generator

        # 4. Filter only the 3-month data (May-24, June-24, July-24)
        three_month_report = full_report[
            (full_report['Month'].isin(['May-24','June-24','July-24']))
        ]

        # Converting to JSON for caching
        three_month_report_json = three_month_report.to_json(orient='records')

        # 5. Store the filtered report in cache
        cache.set('three_month_report', three_month_report_json, timeout=300)

        return jsonify({'message': 'Data uploaded and reports cached successfully'}), 200
    else:
        return jsonify({'error': 'File format not allowed'}), 400

@bp.route('/report/incentive-qty/fetch', methods=['GET'])
def incentive_qty_fetch():
    # 1. Check if user requested subset of months
    requested_months = request.args.getlist('month')
    if not requested_months:
        return jsonify({'error': 'No months specified'}), 400

    # 2. Try retrieving the 3-month chunk from cache
    three_month_report = cache.get('three_month_report')
    if three_month_report is None:
        # 3. If not in cache, check DB or regenerate
        db_data = read_local_data('brand_tieup_1')  # Example function
        if db_data is not None:
            # Recreate the 3-month portion
            three_month_report = db_data[
                (db_data['Month'].isin(['May-24','June-24','July-24']))
            ]
            cache.set('three_month_report', three_month_report, timeout=300)
        else:
            # Full regeneration if not in DB
            # Read brand_tieup_1 data from local
            fresh_data = generate_il_report()
            three_month_report = fresh_data[
                (fresh_data['Month'].isin(['May-24','June-24','July-24']))
            ]
            cache.set('three_month_report', three_month_report, timeout=300)

    # The result would come in json format from cache
    three_month_report = pd.read_json(three_month_report)
    
    # 4. Filter the cached data by requested months
    filtered = three_month_report[three_month_report['Month'].isin(requested_months)]
    return filtered.to_json(orient='records'), 200


Redis connection successful
